# PyTorch code for a Multilayer Perceptron

In this section we will go through the code for a multilayer perceptron in PyTorch.

First of all we set up the required imports and set up the device used for training.

In [20]:
from __future__ import division, print_function, absolute_import
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
      
print('PyTorch version:', torch.__version__, ' Device:', device)

PyTorch version: 1.5.0+cu101  Device: cuda


Here are the relevant network parameters and graph input for context.

In [0]:
# Hyper-Parameters
learning_rate = 0.001 # Initial learning rate
training_epochs = 15 # Number of epochs to train
batch_size = 100 # Number of images per batch
display_step = 1 # How often to output model metrics during training

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

Here, we load the MNIST dataset from the torchvision library.

In [0]:
# The dataset
train_dataset = datasets.MNIST('./data', 
                               train=True, 
                               download=True, 
                               transform=transforms.ToTensor())

test_dataset = datasets.MNIST('./data', 
                                    train=False, 
                                    transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False)

### Model Creation
Here, we create a ‘multi-layer’ model as there is more than one hidden layer, as below we define `fc_1` and `fc_2`.

The MLP definition below does two things:

1. It defines the model in Multilayer_Perceptron().
2. It initialises and assigns values to each layer of the network.

In [0]:
class Multilayer_Perceptron(nn.Module):
    def __init__(self):
        super(Multilayer_Perceptron, self).__init__()

        # Hidden fully connected layers with 256 neurons
        self.fc1 = nn.Linear(n_input, n_hidden_1)
        self.fc2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.out = nn.Linear(n_hidden_2, n_classes)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.out(x), dim=1)

### Define loss and optimizer

In the following snippet we define our model, loss operation, optimiser.

In [0]:
model = Multilayer_Perceptron().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

### Train and evaluate the model

In [27]:
# Set model to training mode
model.train()

# training loop
for epoch in range(training_epochs):
# Loop over each batch from the training set
    for batch_idx, (img, lbl) in enumerate(train_loader):
        # Copy image data to GPU if available
        img = img.to(device)
        lbl = lbl.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass image data through the network
        output = model(img)

        # Calculate loss
        loss = criterion(output, lbl)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
    
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "Loss = {:.9f}".format(loss.item()))

print("Optimization Finished!")

# set model to evaluation mode

model.eval()

correct = 0

for test_img, test_lbl in test_loader:

    test_img = test_img.to(device)
    test_lbl = test_lbl.to(device)

    output = model(test_img)

    _, pred = torch.max(output, dim=1)

    correct += pred.eq(test_lbl).cpu().sum()

accuracy = 100. * correct.to(torch.float32) / len(test_loader.dataset)

print("\nAccuracy:", accuracy.item())


Epoch: 0001 Loss = 0.002095348
Epoch: 0002 Loss = 0.010344102
Epoch: 0003 Loss = 0.013237882
Epoch: 0004 Loss = 0.000267775
Epoch: 0005 Loss = 0.009599728
Epoch: 0006 Loss = 0.016471514
Epoch: 0007 Loss = 0.000801177
Epoch: 0008 Loss = 0.001382481
Epoch: 0009 Loss = 0.010902218
Epoch: 0010 Loss = 0.000892415
Epoch: 0011 Loss = 0.002392501
Epoch: 0012 Loss = 0.011192990
Epoch: 0013 Loss = 0.000859393
Epoch: 0014 Loss = 0.000613048
Epoch: 0015 Loss = 0.037429959
Optimization Finished!

Accuracy: 98.01000213623047
